In [1]:
import os
os.chdir("../../")

In [2]:
## Imports
import pandas as pd


In [56]:
## Load FT meetings
df1 = pd.read_pickle("./data_collection/meetings/ft_meetings_2012-2021.pkl")
df = pd.read_pickle("./data_collection/meetings/ft_meetings_latest_2years.pkl")

In [57]:
df["year"] = df["time"].str[:4].astype(int)
df["month"] = df["time"].str[5:7].astype(int)
print(len(df))
df.head(1)

72551


,speaker,time,text,year,month
0,Formanden Henrik Dam Kristensen,2020-11-25 H13:00,Mødet er åbnet.,2020,11


In [58]:
df = df[df["year"]>2020]
print(len(df))
df.head(1)

61139


,speaker,time,text,year,month
11412,Den fg. formand Christian Juhl,2021-01-12 H13:15,"Forhandlingen er åbnet, og den første, der får...",2021,1


In [59]:
df = df[(df["year"]>2021) | (df["month"]>=7)]
print(len(df))
df.head(1)

21698


,speaker,time,text,year,month
50853,Første næstformand Karen Ellemann,2021-08-26 H13:00,"Forhandlingen er åbnet, og den første ordfører...",2021,8


In [60]:
## Clean speaker col
from utils.functions import load_speaker_fix_dict, load_fix_party_affiliations
import re
pattern, rep = load_speaker_fix_dict()
replace_dict = {'Tredje næstformand':'', 'Anden næstformand':'', 'Skatteministeren':'',
                       'Miljøministeren':'', 'Fødevareministeren':'', 'Statsministeren':'', 'Kirkeministeren':'',
                       'Uddannelses- og forskningsministeren':'', 'Ministeren for sundhed og forebyggelse':'','og kirke':'',
                       'Erhvervs- og vækstministeren':'', 'Undervisningsministeren':'', 'Udenrigsministeren':'',
                       'Den fg. formand':'', 'Forsvarsministeren':'', 'Fjerde næstformand':'', 'Formanden':'',
                       'Handels- og udviklingsministeren': '', 'Transportministeren':'','Udviklingsministeren':'',
                       'Aldersformanden (Bertel Haarder)':'','Aldersformanden':'','Ministeren for ligestilling og kirke':'',
                       'Ministeren for børn, ligestilling, integration og sociale forhold':'','Kulturministeren':'',
                       'Ministeren for by, bolig og landdistrikter':'', 'Ministeren for nordisk samarbejde':'',
                       'Justitsministeren':'', 'Økonomi- og indenrigsministeren':'', 'Finansministeren':'',
                       'Første næstformand':'', 'Beskæftigelsesministeren':'','Ældreministeren':'',
                       'Klima-, energi- og bygningsministeren':'', 'Miljø- og fødevareministeren':'',
                       'Udlændinge-, integrations- og boligministeren':'', 'Social- og indenrigsministeren':'',
                       'Miljø- og fødevareministeren':'','Sundheds- og ældreministeren':'',
                       'Ministeren for børn, undervisning og ligestilling':'','Transport- og bygningsministeren':'', 
                       'Energi-, forsynings- og klimaministeren':'','Børne- og socialministeren':'',
                       'Sundhedsministeren':'','Ministeren for offentlig innovation':'','Erhvervsministeren':'',
                       'Transport-, bygnings- og boligministeren':'','Udlændinge- og integrationsministeren':'',
                       'Ministeren for ligestilling':'','Ministeren for udviklingssamarbejde':'',
                       'Ministeren for fiskeri og ligestilling':'','Social-, børne- og integrationsministeren':'',
                       'Ministeren for forskning, innovation og videregående uddannelser':'',
                       'Handels- og europaministeren':'','Ministeren for fødevarer, fiskeri og ligestilling':'',
                       'Boligministeren':'','Børne- og undervisningsministeren':'',
                        'Klima-, energi- og forsyningsministeren':'','Ministeren for fødevarer, landbrug og fiskeri':'',
                        'Beskæftigelsesminister og minister for ligestilling':'',

                    # also fix other strange things in data
                       '(Bertel Haarder) Bertel Haarder': 'Bertel Haarder (V)',
                       'Hans Christian Schmidt Hans Christian Schmidt':'Hans Christian Schmidt (V)',
                       'Karen Ellemann Karen Ellemann':'Karen Ellemann (V)',
                       'Peter Christensen Peter Christensen':'Peter Christensen (V)', 
                       'Karsten Lauritzen Karsten Lauritzen':'Karsten Lauritzen (V)',
                       'Kristian Jensen Kristian Jensen':'Kristian Jensen (V)',
                       'Søren Pape Poulsen Søren Pape Poulsen':'Søren Pape Poulsen (KF)',
                       'Ole Birk Olesen Ole Birk Olesen':'Ole Birk Olesen (LA)',
                       'Eva Kjer Hansen Eva Kjer Hansen':'Eva Kjer Hansen (V)',
                       'Morten Østergaard Morten Østergaard':'Morten Østergaard (RV)',
                       'Nick Hækkerup Nick Hækkerup':'Nick Hækkerup (S)',
                       'Rasmus Helveg Petersen Rasmus Helveg Petersen':'Rasmus Helveg Petersen (RV)',
                       'Benny Engelbrecht Benny Engelbrecht':'Benny Engelbrecht','Rasmus Prehn Rasmus Prehn':'Rasmus Prehn',
                       'Mette Frederiksen Mette Frederiksen':'Mette Frederiksen','Mogens Jensen Mogens Jensen':'Mogens Jensen',
                       'Morten Bødskov Morten Bødskov':'Morten Bødskov'}


In [61]:
def clean_names2(row):
    match = re.findall(pattern, row)
    if len(match) > 0:
        clean1 = row[len(match[0]):].strip()
        if clean1 == "":
            match = re.findall(pattern, row)
            if len(match) > 0 :
                return re.sub(pattern, replace_dict.get(match[0]), row)
            else: return "MISSING"
        else: return clean1
    else: return row


In [62]:

df["speaker_clean"] = df.apply(lambda row: clean_names2(row["speaker"]), axis=1)

In [63]:
df.head(1)

,speaker,time,text,year,month,speaker_clean
50853,Første næstformand Karen Ellemann,2021-08-26 H13:00,"Forhandlingen er åbnet, og den første ordfører...",2021,8,Karen Ellemann


In [64]:
parties = load_fix_party_affiliations()
def party_affil1(row):
    if parties.get(row) == None:
        return row
    else:
        return row + " " + parties.get(row)

def party_affil2(row):
    return row.split(" ")[-1]

parties.get("Henrik Dam Kristensen")

'(S)'

In [65]:
df["speaker_cleaner"] = df.apply(lambda row: party_affil1(row["speaker_clean"]), axis=1)
df.head(1)

,speaker,time,text,year,month,speaker_clean,speaker_cleaner
50853,Første næstformand Karen Ellemann,2021-08-26 H13:00,"Forhandlingen er åbnet, og den første ordfører...",2021,8,Karen Ellemann,Karen Ellemann (V)


In [66]:
df["party"] = df.apply(lambda row: party_affil2(row["speaker_cleaner"]), axis=1)
df.head(1)

,speaker,time,text,year,month,speaker_clean,speaker_cleaner,party
50853,Første næstformand Karen Ellemann,2021-08-26 H13:00,"Forhandlingen er åbnet, og den første ordfører...",2021,8,Karen Ellemann,Karen Ellemann (V),(V)


In [67]:
df.isnull().sum()

speaker            0
time               0
text               0
year               0
month              0
speaker_clean      0
speaker_cleaner    0
party              0
dtype: int64

In [68]:
def clean_names3(row):
    return row.split("(")[0]

df["speaker_cleaner"] = df.apply(lambda row: clean_names3(row["speaker_cleaner"]), axis=1)
df.head(1)

,speaker,time,text,year,month,speaker_clean,speaker_cleaner,party
50853,Første næstformand Karen Ellemann,2021-08-26 H13:00,"Forhandlingen er åbnet, og den første ordfører...",2021,8,Karen Ellemann,Karen Ellemann,(V)


In [69]:
def get_time(row):
    return row.split(" ")[0].replace("-", "")

df["date"] = df.apply(lambda row: get_time(row["time"]), axis=1)
df["date"] = pd.to_datetime(df['date'], format='%Y%m%d')
df.head(1)

,speaker,time,text,year,month,speaker_clean,speaker_cleaner,party,date
50853,Første næstformand Karen Ellemann,2021-08-26 H13:00,"Forhandlingen er åbnet, og den første ordfører...",2021,8,Karen Ellemann,Karen Ellemann,(V),2021-08-26


In [70]:
df["lower"] = df[["speaker"]].applymap(str.lower)
df.head(1)

,speaker,time,text,year,month,speaker_clean,speaker_cleaner,party,date,lower
50853,Første næstformand Karen Ellemann,2021-08-26 H13:00,"Forhandlingen er åbnet, og den første ordfører...",2021,8,Karen Ellemann,Karen Ellemann,(V),2021-08-26,første næstformand karen ellemann


In [71]:
#drop all forman /facilitator quotes
print(len(df))
df = df[df['lower'].str.contains("formand")==False]
print(len(df))

21698
10491


In [72]:
df_clean = df
df_clean = df_clean[["speaker","speaker_cleaner", "party", "text", "date"]].rename(columns={"speaker_cleaner":"politician"})
df_clean.head(1)

,speaker,politician,party,text,date
50854,Henrik Møller (S),Henrik Møller,(S),Tak for det. Det er jo noget af et syn hernede...,2021-08-26


In [73]:
df_clean.sort_values(by="date", ascending=False).head(5)

,speaker,politician,party,text,date
72546,Jette Gottlieb (EL),Jette Gottlieb,(EL),Mange tak. Hele den her debat handler om noget...,2022-03-16
72468,Torsten Gejl (ALT),Torsten Gejl,(ALT),Hvis man nu bruger 2 mia. kr. fra udviklingsbi...,2022-03-16
72432,Anne Sophie Callesen (RV),Anne Sophie Callesen,(RV),"Vi har ikke, da vi forhandler om loven, altså ...",2022-03-16
72434,Morten Messerschmidt (DF),Morten Messerschmidt,(DF),"Tak, formand. Nu skal jeg jo ikke på den måde ...",2022-03-16
72436,Anne Sophie Callesen (RV),Anne Sophie Callesen,(RV),"Jamen tak for forslaget. Men nej, vi kunne ald...",2022-03-16


In [75]:
#df.to_pickle("ft_meetings_2012-2021_clean.pkl")
df_clean.to_pickle("ft_meetings_2021new-2022_clean_v2_p4.pkl",protocol=4)

In [40]:
df_load = pd.read_pickle("./ft_meetings_2012-2021_clean_v2.pkl")

In [41]:
df_load.head()

,speaker,politician,party,text,date
8,Statsministeren Helle Thorning-Schmidt,Helle Thorning-Schmidt,(S),(Talen er under udarbejdelse) (Talen er under ...,2012-10-02
9,Statsministeren Helle Thorning-Schmidt,Helle Thorning-Schmidt,(S),Vi skal bringe Danmark godt og trygt gennem kr...,2012-10-02
15,Kim Andersen (V),Kim Andersen,(V),Tak for det. Jeg vil gerne spørge erhvervsmini...,2012-10-03
17,Erhvervs- og vækstministeren Ole Sohn,Ole Sohn,Sohn,"Jeg er helt enig med spørgeren i, at vi har en...",2012-10-03
19,Kim Andersen (V),Kim Andersen,(V),"Jeg er helt med på de initiativer, der er tage...",2012-10-03


In [76]:
df1_cl = pd.read_csv("../../climate_classifier/scored_data/ft_meetings_climate_all_2.1.csv")

FileNotFoundError: [Errno 2] No such file or directory: '../../climate_classifier/scored_data/ft_meetings_climate_all_2.1.csv'

In [ ]:
df2_cl = pd.read_

In [2]:
import pandas as pd
df_test = pd.read_pickle("./ft_meetings_2021new-2022_clean_v2.pkl")

In [3]:
df_test

,speaker,politician,party,text,date
50854,Henrik Møller (S),Henrik Møller,(S),Tak for det. Det er jo noget af et syn hernede...,2021-08-26
50856,Victoria Velasquez (EL),Victoria Velasquez,(EL),"Tak for det. Jeg må indrømme, at jeg synes, de...",2021-08-26
50858,Henrik Møller (S),Henrik Møller,(S),"Vi har jo sagt, at vi nu laver den her lønstru...",2021-08-26
50860,Victoria Velasquez (EL),Victoria Velasquez,(EL),Men man kunne jo sagtens stemme for det ændrin...,2021-08-26
50862,Henrik Møller (S),Henrik Møller,(S),"Jamen altså, når vi snakker om parterne, handl...",2021-08-26
...,...,...,...,...,...
72538,Christian Juhl (EL),Christian Juhl,(EL),Nej.,2022-03-16
72540,Morten Messerschmidt (DF),Morten Messerschmidt,(DF),"Jeg blev bare også inspireret af det svar, som...",2022-03-16
72542,Christian Juhl (EL),Christian Juhl,(EL),Jamen det vil jeg gerne gøre. Jeg vil opfordre...,2022-03-16
72544,Jens Rohde (KD),Jens Rohde,(KD),"Tak for det. Jeg har bedt om ordet, fordi jeg ...",2022-03-16
